# Fase #3. Extracción de Detalles de Actores con Selenium

In [ ]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pandas as pd

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from time import sleep

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By  
# By se utiliza para localizar elementos en una página web mediante diferentes métodos como XPATH, CSS_SELECTOR, etc.
import re

In [ ]:
# Lista para almacenar las listas de IDs de película por año
listas_por_año_actores = {}

# Nombres de los archivos CSV por año, esto debe de estar en el escritorio para poder usarlos a menos que los tenga en el Visual
archivos_csv_pelis = ['CSV/datos_2015.csv', 
                        'CSV/datos_2016.csv', 
                        'CSV/datos_2017.csv', 
                        'CSV/datos_2018.csv', 
                        'CSV/datos_2019.csv', 
                        'CSV/datos_2020.csv'] 

# Recorro el diccionario para ir a ver sus datos
for archivo in archivos_csv_pelis:
    peli_actor = re.search(r'\d{4}', archivo).group()
    
    if peli_actor:
        # Leer el archivo CSV
        df = pd.read_csv(archivo)
        
        # Filtrar las filas que tienen valores en la columna 'Id_pelicula', para sacar igual sus nombre que es más seguro con el id
        df_con_id = df.dropna(subset=['Id_pelicula'])
        
        # Crear una lista de los Nom de película accediendo con su ID
        nom_peliculas = df_con_id['Id_pelicula'].tolist()
        
        # Agregar la lista de Nom de película a la lista principal
        listas_por_año_actores[peli_actor] = nom_peliculas
    else:
        print(f"No se pudo encontrar el año en el archivo: {archivo}")

# Imprimir la lista de listas
print(listas_por_año_actores.keys())


In [ ]:
def all_date():
    
    driver = webdriver.Chrome()

    # Abre la página web especificada
    driver.get("https://www.imdb.com/")
    driver.maximize_window()
    sleep(2)

    # Aceptamos las cookies
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
    sleep(2)
    
    # Lista que contiene las peliculas, le indico el año que quiero buscar y que franja o cantidad quiero buscar
    lista_actores_peliculas = listas_por_año_actores['2020'][50:70]
    lista_detalles_actores = [] # Guardo todos los datos que extraigo aqui. 

    # Recorro la lista por ID y empieza a buscar en la pagina
    for id in lista_actores_peliculas:
        
        driver.find_element("css selector", "#suggestion-search").send_keys(id, Keys.ENTER) #Barra de busqueda
        sleep(2)
        
        try:   # clica en los nombres de los actors uno a uno  
            element = driver.find_element("css selector", "[data-testid='title-cast-header'] .ipc-title-link-wrapper")
            driver.execute_script("arguments[0].click();", element) 
        except:
            continue                                 
        sleep(2) 
        # Busco que extraiga solo esa franja de actores, si quiero más lo cambio, y empieza por 2, porque el html o selector de los actores empiza desde 2
        for i in range(2, 5): 
            
            try: # clica en el actor
                driver.find_element("css selector", f"#fullcredits_content > table.cast_list > tbody > tr:nth-child({i}) > td:nth-child(2) > a").click()
            except:
                continue
            
            sleep(2)
                        
            try: # Extraigo nombre del actor                                       
                actores = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/h1/span').text
            except:                                            
                actores = 'No info'
            sleep(2)    
                                                                    
            try: # Extraigo fecha de nacimiento 
                nacimiento = driver.find_element('css selector', '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-92625f35-4.iDcoFI > div.sc-92625f35-6.gHKhNg > div.sc-92625f35-11.eyxQlE > section > aside > div > span:nth-child(2)').text
            except:
                nacimiento = 'No info'
            
            try: # Extraigo sus premios o nominaciones 
                premios = driver.find_element("css selector", '[data-testid="award_information"] span').text   
                #premios = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(8) > div > ul > li > div').text
            except:
                premios = 'No info'
                
            try: # Extraigo que hace en la peli o short. Ej: Direct@r, Escrit@r, Product@r, etc.... 
                que_hace = driver.find_element('css selector', '[data-testid="hero__pageTitle"] ~ ul').text
                que_hace = que_hace.split('\n')
                que_hace = ", ".join(que_hace)
            except:
                que_hace = "No info"
                
            try: # Extraigo las pelis o shor del porque es conocido ese actor y lo voy juntano en el string
                conocido_por = ''                                          
                for c in range(1,4):                                     
                    conocido = driver.find_element('css selector', f'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({c}) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top > a').text
                    conocido_por += conocido + ", "
                conocido_por = conocido_por.rstrip(", ")
            except:
                conocido_por = "No info"
            
            # Guardo todos los datos en tuplas por cada actor y luego en la lista    
            tupla_actores = (actores, nacimiento, premios, que_hace, conocido_por)
            lista_detalles_actores.append(tupla_actores)

            driver.back()   

    return lista_detalles_actores


In [ ]:
# Hago la llamada a la función para sacar los datos y lo guardo en una variable 
full_2020_actores = all_date()

In [ ]:
# Lo muestro en la tabla para ver sus datos con sus respectivos nombres identificativos 
detalles_actores_2020 = pd.DataFrame(full_2020_actores, columns=['Nombre_Actor','Año_nacimiento', 'Premios', 'Qué_hace', 'Conocido_por'])

In [ ]:
# Lo convierto en un CSV para descargarlo
detalles_actores_2020.to_csv('Info_actor_2020.csv', index=False)